In [1]:
# this code plot the distribution of PCAWG sample snv numbers
# Author: Yiyun
import pandas as pd
import os, pickle, time
from tqdm import tqdm
import seaborn as sns
import numpy as np
from multiprocessing import Pool
from collections import Counter

In [2]:
### load file -- pcawg sample info
pcawg_info = '../anno_ref/PCAWG_sample_info.txt'
df_pcawg = pd.read_csv(pcawg_info,sep='\t')
#all gene list
lgene = pickle.load(open('../proc_09152020/gene_name_list_062121.pkl','rb'))# full coverage file

### Mutation bias bins

In [26]:
### Directory for maf file and cohort information
dir_cohorts = '../anno_ref/cohorts'
dir_maf = '../maf_out/maf_cohorts_060121'
feature_type = 'histology'
df = pd.read_csv(os.path.join(dir_cohorts,feature_type+'.csv'))
lfeat = df[feature_type].unique()

# mutation bias bins
exome_fcov = '../anno_ref/exome_full192.coverage.txt'
df_fullcov = pd.read_csv(exome_fcov,sep = '\t')
mutbias_tab = df_fullcov['categ'].unique().tolist()
mutbias_tab = df_fullcov['categ'].unique().tolist()

In [30]:
### Functions of getting contxt and add context into dataframe
def get_ctxt(x):
    mid_idx = int((len(x)-1)/2)
    bidx = mid_idx-1; aidx = mid_idx+1
    bb = x[bidx];ref = x[mid_idx];ab = x[aidx]
    return bb,ref,ab
def get_ctxtdf(feat):   
        global dir_maf, feature_type
        #Read maf files
        maf = os.path.join(dir_maf, feature_type,feat+'.csv')
        df_maf = pd.read_csv(maf, sep = '\t')
        ldonor = df_maf['Donor_ID'].unique().tolist()
        
        # Initialize the dataframe
        df_org = pd.DataFrame(0,columns = mutbias_tab, index = ldonor)
        
        # For each donor, get silent mutation bin
        for donor in ldonor:
            df_donor = df_maf[df_maf['Donor_ID'] == donor]
            df_syn = df_donor[df_donor['Variant_Classification'] == 'Silent']
            ref = df_syn['Reference_Allele']; alt = df_syn['Tumor_Seq_Allele2']; ctxt = df_syn['ref_context']
            old_ctxt = ctxt.apply(get_ctxt)
            # Append the list of bins into a list then convert into a count dictionary
            ltxt_donor = []
            for idx,txt in old_ctxt.items():
                txt_change = f'{txt[0]}({ref[idx]}->{alt[idx]}){txt[2]}'
                ltxt_donor.append(txt_change.upper())
            dict_txt_donor = dict(Counter(ltxt_donor))
            # Append count to dataframe
            for key in dict_txt_donor:
                df_org.loc[donor,key] = dict_txt_donor[key]
        df_org['feature'] = feat 
        print(f'FINISH{feat}')

        return df_org

In [31]:
p = Pool(10)
res = p.map(get_ctxtdf, lfeat)
p.close()
p.join()

/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17,24,28) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


FINISHCNS-Oligo
FINISHCNS-PiloAstro
FINISHThy-AdenoCA


/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17,24) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


FINISHPanc-Endocrine


/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17,24) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,24,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,24,28,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/s

...

FINISHBreast-AdenoCA
FINISHPanc-AdenoCA
FINISHStomach-AdenoCA
FINISHLung-SCC


/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,3,17,24,28,30,33,34,35,36,40) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


FINISHEso-AdenoCA


/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


FINISHColoRect-AdenoCA
FINISHLiver-HCC
FINISHSkin-Melanoma


In [32]:
### Concat and calculate the final dataframe
df_mutbias = pd.concat(res)
# mutbias_tab.append('feature')
df_mutbias = df_mutbias[mutbias_tab]
# df_mutbias.to_csv('./figure1/mut_bins.csv')

Organize mutbias data for plot

In [22]:
### Combine mutation bins
df_mutbias = pd.read_csv('./figure1/mut_bins.csv', index_col = 0)

In [27]:
### Determine transition, transversion, transition
from utils import *

dict_mutname={6:'AT_transversion',5:'AT_transition',4:'GC_transversion',\
              3:'GC_transition',2:'CpG_transversion',1:'CpG_transition'}

dict_melt = {}
for mut in mutbias_tab[:-1]:
    bases = re.findall(r'[A-Z]+', mut)
    melt_categ = dict_mutname[validate_categ(bases[1], bases[2],bases[0]+bases[1]+bases[3])]
    if melt_categ not in dict_melt:
        dict_melt[melt_categ] = []
    dict_melt[melt_categ].append(mut)

### Combine the mutbias dictionary into 6 mutation categories
df_mutcateg = pd.DataFrame()
for k,v in dict_melt.items():
    df_temp = df_mutbias[v].sum(axis=1).to_frame(name = k)
    df_mutcateg = pd.concat([df_mutcateg, df_temp], axis = 1)
df_mutcateg = pd.concat([df_mutcateg,df_mutbias['feature']], axis=1)

df_categperc = \
(100. * df_mutcateg.iloc[:,:-1].transpose() / df_mutcateg.iloc[:,:-1].transpose().sum()).round(2).transpose()
df_categperc = pd.concat([df_categperc,df_mutbias['feature']], axis=1)
# # df_categperc.to_csv('./figure1/mut_categs.csv')

### Get mutation number

In [4]:
### Directory for maf file and cohort information
dir_cohorts = '../anno_ref/cohorts'
dir_maf = '../maf_out/maf_cohorts_060121'
feature_type = 'histology'
df = pd.read_csv(os.path.join(dir_cohorts,feature_type+'.csv'))
lfeat = df[feature_type].unique()

lgene = pickle.load(open('../proc_09152020/gene_name_list_062121.pkl','rb'))

In [5]:
### Iterate through organs to get mutation number
def get_n_mut(feat):
    global dir_maf, feature_type
    #Read maf files
    maf = os.path.join(dir_maf, feature_type,feat+'.csv')
    df_maf = pd.read_csv(maf, sep = '\t')
    df_maf = df_maf[df_maf['Hugo_Symbol'].isin(lgene.tolist())]
    ldonor = df_maf['Donor_ID'].unique().tolist()
    df_maf = df_maf.set_index(['Donor_ID','Variant_Classification'])
    
    df_temp = pd.DataFrame(index = ldonor, columns = ['n_silent','feature'])
    
    for donors in ldonor:
        try:
            df_donor = df_maf.loc[(donors,'Silent'),:]
            n_syn = len(df_donor)
            df_temp.loc[donors,:] =[n_syn,feat] 
        except KeyError:
            df_temp.loc[donors,:] =[0,feat] 
    
    return df_temp

p = Pool(8)
res_mut = p.map(get_n_mut, lfeat)
p.close()
p.join()

/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-bzb/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17,24) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-bzb/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-bzb/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17,24) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-bzb/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-bzb/lib/python3.8/

AttributeError: 'list' object has no attribute 'tolist'

In [113]:
### Concat and calculate the final dataframe
df_all = pd.concat(res_mut, ignore_index = False)
# df_all.to_csv('mut_hete_patients_May-21-2021.csv')
df_all.to_csv('./figure1/mut_hete_patients_silent.csv')

***
Analysis - the median, the max and the min

In [6]:
df_all = pd.read_csv('./figure1/mut_hete_patients_silent.csv')
df_all['syn_freq'] = df_all['n_silent']/float(7564562)*float(1000000)

In [9]:
### Get median mutation frequency
group = df_all[['feature','syn_freq']].groupby(by = ['feature'])
group.median().sort_values(by = 'syn_freq').to_csv('./figure1/median_mut_freq.csv')

In [18]:
### Get the max and min mutation frequency within the cohorts
df_stat = pd.concat([group.max(),group.min(),group.median()],axis = 1)
df_stat.columns = ['max_synfreq','min_synfreq','median_synfreq']
df_stat['max-min'] = df_stat['max_synfreq'] - df_stat['min_synfreq']

In [20]:
df_stat.to_csv('./figure1/stat_synmut_freq.csv')

In [21]:
df_stat

,max_synfreq,min_synfreq,median_synfreq,max-min
feature,,,,
Biliary-AdenoCA,103.376772,0.264391,1.850735,103.112381
Bladder-TCC,24.059556,0.925368,5.552205,23.134188
Bone-Benign,0.132195,0.132195,0.132195,0.000000
Bone-Cart,0.793172,0.132195,0.396586,0.660977
Bone-Epith,1.189758,0.264391,0.462684,0.925368
Bone-Osteoblast,0.264391,0.000000,0.132195,0.264391
Bone-Osteosarc,4.494642,0.132195,1.057563,4.362447
Breast-AdenoCA,34.767380,0.132195,1.982930,34.635184
Breast-DCIS,1.321954,0.264391,0.528781,1.057563


Olde code

In [ ]:
#             n_mis = len(df_donor[df_donor['Variant_Classification'] == 'Missense_Mutation'])
#             n_ncod = len(df_donor[df_donor['Variant_Classification'].isin(['Intron',"3'UTR","5'UTR"])])
#             n_nsyn = len(df_donor[df_donor['Variant_Classification'].isin(['Missense_Mutation', 'Nonsense_Mutation','Splice_Site', 'Nonstop_Mutation','Start_Codon_SNP'])])

#             l_donor = [donors, n_syn,n_mis,n_nsyn,n_ncod]
#             df_temp = pd.DataFrame([l_donor], columns = ['Donor','n_silent','n_missense','n_nsyn','n_noncoding'])
#             df_org = pd.concat([df_org,df_temp], ignore_index = True)

### Pie plot data

In [22]:
### Directory for maf file and cohort information
dir_cohorts = '../anno_ref/cohorts'
dir_maf = '../maf_out/maf_cohorts_060121'
feature_type = 'histology'
df = pd.read_csv(os.path.join(dir_cohorts,feature_type+'.csv'))
lfeat = df[feature_type].unique()

lgene = pickle.load(open('../proc_09152020/gene_name_list_062121.pkl','rb'))

In [30]:
### Count all synonymous and missense mutations and other mutations
### Iterate through organs to get mutation number
df_maf = pd.read_csv(os.path.join(dir_maf,feature_type, 'Ovary-AdenoCA'+'.csv'), sep = '\t')
mut_class = df_maf['Variant_Classification'].unique()

def get_n_mut(feat):
    global dir_maf, mut_class
    
    df_tot_org = pd.DataFrame(columns = mut_class, index = [feat])
    #Read maf files
    df_maf = pd.read_csv(os.path.join(dir_maf,feature_type, feat+'.csv'), sep = '\t')
    df_maf = df_maf[df_maf['Hugo_Symbol'].isin(lgene)]
    # Get number of mutations in each categories
    for mut_type in mut_class:
        n_mut = len(df_maf[df_maf['Variant_Classification'] == mut_type])
        df_tot_org.loc[feat,mut_type] = n_mut
    return df_tot_org

In [31]:
p = Pool(8)
res = p.map(get_n_mut, lfeat)
p.close()
p.join()

/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17,24) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17,24) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (1,17,24,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py:108: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-k

In [32]:
df_tot = pd.concat(res)
df_tot.loc['sum',:] = df_tot.sum()

In [34]:
### All coding mutations
labels = ['Missense','Silent','Other']
n_mis = df_tot.loc['sum','Missense_Mutation']
n_syn = df_tot.loc['sum','Silent']
n_other = df_tot.loc['sum','Nonsense_Mutation']+df_tot.loc['sum','Splice_Site']\
    +df_tot.loc['sum','De_novo_Start_OutOfFrame']+df_tot.loc['sum','Frame_Shift_Del']\
+df_tot.loc['sum','In_Frame_Del']+df_tot.loc['sum','Frame_Shift_Ins']+df_tot.loc['sum','De_novo_Start_InFrame']\
+df_tot.loc['sum','Nonstop_Mutation']+df_tot.loc['sum','Start_Codon_Del']+df_tot.loc['sum','Start_Codon_SNP']\
+df_tot.loc['sum','In_Frame_Ins']+df_tot.loc['sum','Stop_Codon_Del']

sizes = [n_mis,n_syn,n_other]

In [36]:
pickle.dump(sizes, open('./figure1/pie_size.pkl','wb'))

### Local mut rate

In [1]:
import pandas as pd
import os,pickle
from tqdm import tqdm
from utils import *
from multiprocessing import Pool
from datetime import date

today = date.today()

In [2]:
dir_cohorts = '../anno_ref/cohorts'
dir_maf = '../maf_out/maf_cohorts_060121'
feature_type = 'histology'
df = pd.read_csv(os.path.join(dir_cohorts,feature_type+'.csv'))
lfeat = df[feature_type].unique()

In [3]:
### Load annotation data
dict_transcript_info = pickle.load(open('../proc_09152020/dict_transcript_info_062121.pkl','rb'))
dict_record = pickle.load(open('../proc_09152020/dict_record_new.pkl','rb'))

In [7]:
### Parameter card
chromosome = '3'
feat = 'Ovary-AdenoCA'
start_range = 27; end_range=77

In [8]:
interval = 1000000
### Create breakpoints for chr1
l_bk = []
breakpoints = list(range(0, len(dict_record['chr'+chromosome]), interval))
for i in range(0, len(breakpoints)-1):
    group = [breakpoints[i], breakpoints[i+1]]
    l_bk.append(group)

In [53]:
### Define function to calculate
def get_chr_mut(i):
    global chromosome, feat
    
    df = pd.DataFrame(columns = ['Donor','chr-pos','n_syn','n_nsyn','n_coding','n_all'])
    df_maf = pd.read_csv(os.path.join(dir_maf,feature_type, feat+'.csv'), sep = '\t')
    ldonor = df_maf['Donor_ID'].unique().tolist()

    for donors in tqdm(ldonor):
#             # Initialize a donor dataframe
        df_donor = df_maf[df_maf['Donor_ID'] == donors]
        df_donor = df_donor[df_donor['Chromosome'] == chromosome]
        df_donor = df_donor[df_donor['Variant_Type'] == 'SNP']
        df_donor = df_donor[(df_donor['Start_position'] < l_bk[i][1]) & \
                            (df_donor['Start_position'] > l_bk[i][0])]
        n_all = len(df_donor)
        n_syn = len(df_donor[df_donor['Variant_Classification'] == 'Silent'])
        n_nsyn = len(df_donor[df_donor['Variant_Classification'].isin(['Missense_Mutation', 'Nonsense_Mutation','Splice_Site', 'Nonstop_Mutation','Start_Codon_SNP'])])
        l_donor = [donors, '1-'+str(i), n_syn, n_nsyn, n_syn+n_nsyn, n_all]
        df_temp = pd.DataFrame([l_donor], columns = ['Donor','chr-pos','n_syn','n_nsyn','n_coding','n_all'])
        df = pd.concat([df,df_temp], ignore_index = True)
        
    return df

In [54]:
### Run get mutation, we don't get the full range
p = Pool(8)
res = p.map(get_chr_mut, range(start_range, end_range))
p.close()
p.join()

### Concat the results
df_all = pd.concat(res, ignore_index = True)

### Save results
fname = feat+'_chr'+chromosome+'_'+str(interval)+'_'+str(start_range)+'-'+str(end_range)+'_'+today.strftime("%b-%d-%Y")+'.csv'
df_all.to_csv(os.path.join('./figure1',fname))

KeyboardInterrupt: 

Process ForkPoolWorker-60:
Process ForkPoolWorker-54:
Process ForkPoolWorker-59:
Process ForkPoolWorker-61:
Process ForkPoolWorker-56:
Traceback (most recent call last):
Process ForkPoolWorker-58:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/aci/sw/anaconda3/2020.07_gcc-4.8.5-khh/lib/python3.8/multiprocessing/process.py", line 315